# Capstone Project - San Francisco Housing Sales Price 
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project, I'll analyze each neighborhood in San Francisco and try to find a relationship between **San Francisco Housing Sales Price** and **nearby venues**. 

Specifically, this report will be targeted to stakeholders interested in investing a **real estate in San Francisco**, helping them to choose the regions with their favorite venues or lower real estate costs.

I'll create a map with each neighborhood in San Francisco segmented and clustered according to housing sales prices and nearby venues.

## Data <a name="data"></a>

In consider of our problem, I found following data sets:
* I found **Median Value Per Squre foot** data of each neighborhood in San Francisco, from Apr 1996 to Jul 2019. The csv file contains the **Region Name, City, CountyName, SizeRank and median_value_per_sqft** during this time period in USD.
* I used **Forsquare API** to get the most common venues of given neighborhood of San Francisco.
* I used **Google Maps API geocoding** to get the center cooridnates of each neighborhood.

In [1]:
import pandas as pd
import numpy as np
import folium 
from geopy.geocoders import Nominatim

Read *Median Value Per Squre foot* data into a pandas DataFrame, it contains Median Value per sqft of each major city in the US from 1996 to 2019, represented by zip code. Drop useless columns and rename *ReginName* to *ZipCode*. We are mainly focusing on the San Francisco data.

In [2]:
df = pd.read_csv('Zip_MedianValuePerSqft_AllHomes.csv',encoding = "ISO-8859-1")
df_sf = df[df['City']=='San Francisco'].reset_index(drop=True)
df_sf.drop(['City','State','Metro','CountyName','SizeRank'],1,inplace=True)
df_sf.rename(columns={'RegionName': 'ZipCode'}, inplace=True)
df_sf.head()

,RegionID,ZipCode,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,97564,94109,268.0,268.0,269.0,270.0,270.0,271.0,273.0,275.0,...,1184,1180,1176,1174,1168,1159,1153,1149,1151,1155
1,97565,94110,184.0,184.0,184.0,184.0,184.0,185.0,186.0,187.0,...,1192,1187,1185,1185,1180,1173,1169,1165,1163,1162
2,97576,94122,187.0,188.0,189.0,189.0,190.0,191.0,192.0,193.0,...,989,988,988,990,988,982,979,980,981,982
3,97567,94112,163.0,164.0,164.0,164.0,164.0,164.0,164.0,165.0,...,863,869,873,878,878,874,872,873,874,877
4,97569,94115,260.0,260.0,260.0,260.0,260.0,260.0,261.0,262.0,...,1221,1221,1221,1221,1214,1205,1198,1197,1202,1207


We are mainly looking into the house value data of 2019, calculate the mean value of each month in 2019.

In [3]:
col_2019 = [col for col in df_sf.columns if '2019' in col]
mean_2019 = df_sf[col_2019].mean(axis=1).to_frame(name='avgPrice_2019')
mean_2019.head()

,avgPrice_2019
0,1158.428571
1,1171.000000
2,983.142857
3,875.142857
4,1206.285714


Insert mean value of 2019 into the dataframe and drop data from other years.

In [4]:
df_sf.drop(df_sf.iloc[:,2:],1,inplace = True)
df_sf['avgPrice_2019'] = mean_2019['avgPrice_2019']
df_sf.head()

,RegionID,ZipCode,avgPrice_2019
0,97564,94109,1158.428571
1,97565,94110,1171.000000
2,97576,94122,983.142857
3,97567,94112,875.142857
4,97569,94115,1206.285714


Use **geopy.geocoders** to build a function and obtain geographical coordinates of each zip code.

In [5]:
def find_coordinates(zipcode):
    add = zipcode
    geolocator = Nominatim(user_agent="ca_explorer")
    location = geolocator.geocode(add)
    latitude = location.latitude
    longitude = location.longitude
    return [latitude, longitude]


Write coordinates information of each zip code into the dataframe and clean the data.

In [6]:
coordinates = []
for item in df_sf['ZipCode']:
    try: 
        coordinates.append(find_coordinates(item))
    except:
        coordinates.append([np.nan, np.nan])
        print('No coordinates found for {}.'.format(item))
coor = pd.DataFrame(coordinates,columns = ['latitude' , 'longitude'])
df_sf[['latitude' , 'longitude']] = coor[['latitude' , 'longitude']]
df_sf_cleaned = df_sf.dropna()
df_sf_cleaned.ZipCode = df_sf_cleaned.ZipCode.astype(str)
df_sf_cleaned.drop('RegionID',1,inplace=True)
df_sf_cleaned.head()

No coordinates found for 94123.
No coordinates found for 94131.


C:\Keren\Software\Anaconda\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Keren\Software\Anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,ZipCode,avgPrice_2019,latitude,longitude
0,94109,1158.428571,37.794083,-122.420519
1,94110,1171.000000,37.753297,-122.416533
2,94122,983.142857,37.759897,-122.473650
3,94112,875.142857,37.723100,-122.444374
4,94115,1206.285714,37.784065,-122.435235


Use **Beautiful Soup** library to scrape table from a website which contains the **Neighborhood Name** and **Population** of each Zip Code.

In [7]:
import requests
from bs4 import BeautifulSoup

In [8]:
url = "http://www.healthysf.org/bdi/outcomes/zipmap.htm"
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')

In [9]:
table = soup.find_all('table')[3]
DF = pd.read_html(str(table))[0]
DF.drop([0,22],inplace=True)
DF.rename(columns={0: "ZipCode", 1: "Neighborhood", 2:"Population"},inplace=True)
DF = DF.astype(str)
DF.head()

,ZipCode,Neighborhood,Population
1,94102,Hayes Valley/Tenderloin/North of Market,28991
2,94103,South of Market,23016
3,94107,Potrero Hill,17368
4,94108,Chinatown,13716
5,94109,Polk/Russian Hill (Nob Hill),56322


Now we have a dataframe wchich contains all geographical data of each neighborhood in San Francisco.

In [10]:
DF_SF = pd.merge(DF, df_sf_cleaned, on='ZipCode')
DF_SF.head()

,ZipCode,Neighborhood,Population,avgPrice_2019,latitude,longitude
0,94102,Hayes Valley/Tenderloin/North of Market,28991,1088.857143,37.779450,-122.418183
1,94103,South of Market,23016,1061.142857,37.774368,-122.411109
2,94107,Potrero Hill,17368,1109.142857,37.792606,-122.407668
3,94108,Chinatown,13716,1197.142857,37.791077,-122.406539
4,94109,Polk/Russian Hill (Nob Hill),56322,1158.428571,37.794083,-122.420519


Create a map of San Francisco with markers of each neighborhood.

In [11]:
sf_coor = find_coordinates('San Francisco')
# create map of San Francisco using latitude and longitude values
map_sf = folium.Map(location=sf_coor, zoom_start=12)

# add markers to map
for lat, lng, neighborhood, Pop, price in zip(DF_SF['latitude'], DF_SF['longitude'], DF_SF['Neighborhood'], DF_SF['Population'], DF_SF['avgPrice_2019']):
    label = '{}, Population: {}, Mean price per sqft: {}'.format(neighborhood, Pop, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)  
    
map_sf

## Methodology <a name="methodology"></a>

Now use Foursquare AIP to find nearby venues data of each neighborhood. First define Foursquare Credentials and Version.

In [12]:
CLIENT_ID = 'BAHTLSRWTZBXVZDJ1BCYG0QGLCTMFET1GIYW40FEZDXKM15R' # your Foursquare ID
CLIENT_SECRET = 'Q0WDT2JIJ4ACQK4SGRCKILHEQG3HG35AU5OR12Q5EEFEEDTH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BAHTLSRWTZBXVZDJ1BCYG0QGLCTMFET1GIYW40FEZDXKM15R
CLIENT_SECRET:Q0WDT2JIJ4ACQK4SGRCKILHEQG3HG35AU5OR12Q5EEFEEDTH


Create a function to process all the neighborhoods in San Francisco.

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called sf_venues.

In [14]:
sf_venues = getNearbyVenues(names=DF_SF['Neighborhood'],
                                   latitudes=DF_SF['latitude'],
                                   longitudes=DF_SF['longitude']
                                  )

Hayes Valley/Tenderloin/North of Market
South of Market
Potrero Hill
Chinatown
Polk/Russian Hill (Nob Hill)
Inner Mission/Bernal Heights
Ingelside-Excelsior/Crocker-Amazon
Castro/Noe Valley
Western Addition/Japantown
Parkside/Forest Hill
Haight-Ashbury
Inner Richmond
Outer Richmond
Sunset
Bayview-Hunters Point
St. Francis Wood/Miraloma/West Portal
Lake Merced
North Beach/Chinatown
Visitacion Valley/Sunnydale


Check the size of the resulting dataframe.

In [15]:
print(sf_venues.shape)
sf_venues.head()

(1120, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hayes Valley/Tenderloin/North of Market,37.77945,-122.418183,Asian Art Museum,37.780178,-122.416505,Art Museum
1,Hayes Valley/Tenderloin/North of Market,37.77945,-122.418183,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
2,Hayes Valley/Tenderloin/North of Market,37.77945,-122.418183,Herbst Theater,37.779548,-122.420953,Concert Hall
3,Hayes Valley/Tenderloin/North of Market,37.77945,-122.418183,Philz Coffee,37.781433,-122.417073,Coffee Shop
4,Hayes Valley/Tenderloin/North of Market,37.77945,-122.418183,War Memorial Opera House,37.778601,-122.420816,Opera House


Get how many venus were returned for each neighborhood.

In [16]:
venues_count = sf_venues.groupby('Neighborhood').count()

Find out how many unique categories can be curated from all the returned venues.

In [17]:
print('There are {} uniques categories.'.format(len(sf_venues['Venue Category'].unique())))

There are 239 uniques categories.


### Analyze Each Neighborhood
Use one hot encoding to transforme all the catagorical data into numerical numbers.

In [18]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Tuscan Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Get the shape of the new one hot dataframe.

In [19]:
sf_onehot.shape

(1120, 239)

Group rows by neighborhood and by taking the mean of the frequency of occerence of each catagory.

In [20]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Tuscan Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Bayview-Hunters Point,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.0,0.000000
1,Castro/Noe Valley,0.02,0.000000,0.02,0.0,0.000000,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000000,0.000000,0.0,0.00,0.020000,0.00,0.0,0.000000
2,Chinatown,0.01,0.010000,0.00,0.0,0.010000,0.0,0.0,0.0,0.01,...,0.0,0.0,0.000000,0.000000,0.0,0.01,0.000000,0.00,0.0,0.000000
3,Haight-Ashbury,0.00,0.014706,0.00,0.0,0.014706,0.0,0.0,0.0,0.00,...,0.0,0.0,0.014706,0.014706,0.0,0.00,0.014706,0.00,0.0,0.014706
4,Hayes Valley/Tenderloin/North of Market,0.00,0.000000,0.00,0.0,0.020000,0.0,0.0,0.0,0.00,...,0.0,0.0,0.030000,0.040000,0.0,0.00,0.000000,0.01,0.0,0.000000


Print each neighborhood along with the top 3 most common venues.

In [21]:
num_top_venues = 3

for hood in sf_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bayview-Hunters Point----
           venue  freq
0       Mountain  0.29
1  Moving Target  0.14
2    Bus Station  0.14


----Castro/Noe Valley----
            venue  freq
0         Gay Bar  0.12
1            Park  0.06
2  Scenic Lookout  0.04


----Chinatown----
         venue  freq
0     Boutique  0.06
1        Hotel  0.05
2  Coffee Shop  0.04


----Haight-Ashbury----
                    venue  freq
0  Thrift / Vintage Store  0.07
1          Clothing Store  0.07
2                Boutique  0.07


----Hayes Valley/Tenderloin/North of Market----
          venue  freq
0  Cocktail Bar  0.05
1          Café  0.04
2   Coffee Shop  0.04


----Ingelside-Excelsior/Crocker-Amazon----
                venue  freq
0         Bus Station  0.12
1  Light Rail Station  0.12
2            Bus Line  0.06


----Inner Mission/Bernal Heights----
                       venue  freq
0         Mexican Restaurant  0.11
1                Coffee Shop  0.05
2  Latin American Restaurant  0.03


----Inner Richmond---

Create the new dataframe and display the top 3 venues for each neighborhood.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bayview-Hunters Point,Mountain,Bus Station,Breakfast Spot
1,Castro/Noe Valley,Gay Bar,Park,Scenic Lookout
2,Chinatown,Boutique,Hotel,Coffee Shop
3,Haight-Ashbury,Clothing Store,Boutique,Thrift / Vintage Store
4,Hayes Valley/Tenderloin/North of Market,Cocktail Bar,Theater,Coffee Shop


## Clustering neighborhood
Import KMeans library

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

Merge the venues catagorical data with mean price, population and venue cont data.

In [25]:
try:
    venues_count.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue Latitude','Venue Longitude', 'Venue Category'],1,inplace = True)
except:
    None

In [26]:
df_merge = DF_SF.drop(['ZipCode','latitude','longitude'],1)
df_merge = df_merge.join(venues_count, on='Neighborhood')
df_merge = df_merge.join(sf_grouped.set_index('Neighborhood'), on='Neighborhood')
df_merge['Population'] = df_merge['Population'].astype(float)/max(df_merge['Population'].astype(float))
df_merge['avgPrice_2019']= df_merge['avgPrice_2019'].astype(float)/max(df_merge['avgPrice_2019'].astype(float))
df_merge['Venue'] = df_merge['Venue']/max(df_merge['Venue'])
df_merge.head()

,Neighborhood,Population,avgPrice_2019,Venue,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,...,Tuscan Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Hayes Valley/Tenderloin/North of Market,0.388447,0.832187,1.0,0.00,0.00,0.00,0.00,0.02,0.0,...,0.0,0.0,0.03,0.04,0.00,0.00,0.00,0.01,0.00,0.0
1,South of Market,0.308389,0.811006,1.0,0.00,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,0.00,0.02,0.01,0.00,0.03,0.00,0.00,0.0
2,Potrero Hill,0.232712,0.847691,1.0,0.00,0.00,0.00,0.00,0.02,0.0,...,0.0,0.0,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.0
3,Chinatown,0.183779,0.914947,1.0,0.01,0.01,0.00,0.00,0.01,0.0,...,0.0,0.0,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.0
4,Polk/Russian Hill (Nob Hill),0.754653,0.885359,1.0,0.00,0.00,0.01,0.01,0.03,0.0,...,0.0,0.0,0.00,0.03,0.00,0.00,0.02,0.02,0.01,0.0


**Run k-means to cluster the neighborhood into 5 clusters.**

In [27]:
# set number of clusters
kclusters = 5

#sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)
sf_grouped_clustering = df_merge.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 3, 2, 1, 0, 2])

Creat a new dataframe that includes the cluster as well as the top 3 venues for each neighborhood.

In [28]:
try:
    neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop('Cluster Labels', 1)
except:
    None
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sf_merged = DF_SF

# merge sf_grouped with sf_data to add latitude/longitude for each neighborhood
#sf_merged.insert(0, 'Cluster Labels', kmeans.labels_)
sf_merged = sf_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
sf_merged = sf_merged.join(venues_count, on='Neighborhood')

sf_merged.head() # check the last columns!

,ZipCode,Neighborhood,Population,avgPrice_2019,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Venue
0,94102,Hayes Valley/Tenderloin/North of Market,28991,1088.857143,37.779450,-122.418183,3,Cocktail Bar,Theater,Coffee Shop,100
1,94103,South of Market,23016,1061.142857,37.774368,-122.411109,4,Nightclub,Cocktail Bar,Coffee Shop,100
2,94107,Potrero Hill,17368,1109.142857,37.792606,-122.407668,2,Hotel,Bubble Tea Shop,Coffee Shop,100
3,94108,Chinatown,13716,1197.142857,37.791077,-122.406539,0,Boutique,Hotel,Coffee Shop,100
4,94109,Polk/Russian Hill (Nob Hill),56322,1158.428571,37.794083,-122.420519,1,Cosmetics Shop,Deli / Bodega,Italian Restaurant,100


## Analysis <a name="analysis"></a>
import libraries for data visualization

In [29]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Create a map with clusters in neighborhood of San Francisco.

In [30]:
# create map
map_clusters = folium.Map(location=sf_coor, zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['latitude'], sf_merged['longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [31]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[1,2,3] + list(range(6, sf_merged.shape[1]))]]

,Neighborhood,Population,avgPrice_2019,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Venue
3,Chinatown,13716,1197.142857,0,Boutique,Hotel,Coffee Shop,100
7,Castro/Noe Valley,30574,1308.428571,0,Gay Bar,Park,Scenic Lookout,50
10,Haight-Ashbury,38738,1177.000000,0,Clothing Store,Boutique,Thrift / Vintage Store,68
14,Bayview-Hunters Point,33170,719.571429,0,Mountain,Bus Station,Breakfast Spot,7
16,Lake Merced,26291,890.571429,0,Yoga Studio,Burger Joint,Intersection,12
18,Visitacion Valley/Sunnydale,40134,776.571429,0,Playground,Pool,Trail,4


Cluster 2

In [32]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[1,2,3] + list(range(6, sf_merged.shape[1]))]]

,Neighborhood,Population,avgPrice_2019,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Venue
4,Polk/Russian Hill (Nob Hill),56322,1158.428571,1,Cosmetics Shop,Deli / Bodega,Italian Restaurant,100
11,Inner Richmond,38939,1140.571429,1,Park,Field,Trail,16
12,Outer Richmond,42473,988.285714,1,Café,Pizza Place,Convenience Store,51
15,St. Francis Wood/Miraloma/West Portal,20624,1040.857143,1,Chinese Restaurant,Coffee Shop,Pizza Place,55


Cluster 3

In [33]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1,2,3] + list(range(6, sf_merged.shape[1]))]]

,Neighborhood,Population,avgPrice_2019,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Venue
2,Potrero Hill,17368,1109.142857,2,Hotel,Bubble Tea Shop,Coffee Shop,100
5,Inner Mission/Bernal Heights,74633,1171.000000,2,Mexican Restaurant,Coffee Shop,Record Shop,100
9,Parkside/Forest Hill,42958,984.000000,2,Bus Stop,Chinese Restaurant,Martial Arts Dojo,27
17,North Beach/Chinatown,26827,1226.000000,2,Italian Restaurant,Chinese Restaurant,Pizza Place,100


Cluster 4

In [34]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[1,2,3] + list(range(6, sf_merged.shape[1]))]]

,Neighborhood,Population,avgPrice_2019,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Venue
0,Hayes Valley/Tenderloin/North of Market,28991,1088.857143,3,Cocktail Bar,Theater,Coffee Shop,100
6,Ingelside-Excelsior/Crocker-Amazon,73104,875.142857,3,Light Rail Station,Bus Station,Bus Line,17


Cluster 5

In [35]:
sf_merged.loc[sf_merged['Cluster Labels'] == 4, sf_merged.columns[[1,2,3] + list(range(6, sf_merged.shape[1]))]]

,Neighborhood,Population,avgPrice_2019,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Venue
1,South of Market,23016,1061.142857,4,Nightclub,Cocktail Bar,Coffee Shop,100
8,Western Addition/Japantown,33115,1206.285714,4,Gift Shop,Tea Room,Bakery,99
13,Sunset,55492,983.142857,4,Coffee Shop,Tailor Shop,Hotpot Restaurant,14


## Conclusion <a name="conclusion"></a>

Now we have segmented all neighborhoods in San Francisco into 5 Clusters based on the average housing price, population and venue numbers and most common venue in each neighborhood. The same labeled neighborhoods are those have most similar house price, population and venues. The stakeholders will be able to know which cluster of neighborhood to invest, considering their financial status and preference of venues.